Model Progress는 학습 중, 이후에 모두 저장될 수 있다. 이는 곧 모델은 긴 학습 시간 중간에 멈춘 곳에서 다시 시작될 수 있다는 것을 나타낸다. 

모델을 저장을 통해서 모델을 공유할 수 있고, 다른 작업에 재활용할 수도 있다. 연구 모델이나 기술들을 공개할 때, 대부분의 머신러닝 엔지니어들은 다음을 공유한다.

- 모델을 생성 하는 코드 
- 학습된 weights 나 parameters 

Tensorflow model을 저장하는 방법은 사용한 API에 따라서 달라질 수 있다. 여기서는 tf.keras를 사용한 예제를 다룰 것이다. 

## <U>Setup</U>

### Install and imports

In [1]:
!pip install -q h5py pyyaml

Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/lib/python3.6/dist-packages/yaml'
Consider using the `--user` option or check the permissions.



### Get an example dataset

In [2]:
from __future__ import absolute_import, division, print_function

import os 

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.10.0'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Define a model

In [4]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                 loss=tf.keras.losses.sparse_categorical_crossentropy,
                 metrics=['accuracy'])
    return model

model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## <U>Save checkpoints during training</U>

주요한 Use case 는 학습 도중과 끝에 자동적으로 checkpoints를 저장하는 것이다. 이 방법을 사용하면 retrain 없이 학습된 모델을 사용할 수 있고, 학습 과정이 갑자기 중단된 경우에도 이어서 학습을 진행할 수 있다.

### Checkpoint callback usage

In [5]:
checkpoint_path = "training_1/cp.cpkt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)

model = create_model()

model.fit(train_images, train_labels, epochs=10,
         validation_data = (test_images, test_labels),
         callbacks= [cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 3s 3ms/step - loss: 1.1855 - acc: 0.6530 - val_loss: 0.7320 - val_acc: 0.7810

Epoch 00001: saving model to training_1/cp.cpkt
Epoch 2/10
1000/1000 [==============================] - 0s 193us/step - loss: 0.4333 - acc: 0.8790 - val_loss: 0.5617 - val_acc: 0.8290

Epoch 00002: saving model to training_1/cp.cpkt
Epoch 3/10
1000/1000 [==============================] - 0s 205us/step - loss: 0.2888 - acc: 0.9310 - val_loss: 0.4906 - val_acc: 0.8410

Epoch 00003: saving model to training_1/cp.cpkt
Epoch 4/10
1000/1000 [==============================] - 0s 203us/step - loss: 0.2045 - acc: 0.9440 - val_loss: 0.4421 - val_acc: 0.8600

Epoch 00004: saving model to training_1/cp.cpkt
Epoch 5/10
1000/1000 [==============================] - 0s 204us/step - loss: 0.1546 - acc: 0.9690 - val_loss: 0.4414 - val_acc: 0.8500

Epoch 00005: saving model to training_1/cp.cpkt
Epoch 6/10
1000/1000 [=======

In [6]:
!ls {checkpoint_dir}

checkpoint  cp.cpkt.data-00000-of-00001  cp.cpkt.index


새로운 Untrained model을 생성하자. Only weights 로부터 모델을 복원할 때, 반드시 original model과 동일한 모델을 사용해야한다. 

In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("untrained model, accuracy : {:5.2}%".format(100*acc))

1000/1000 [==============================] - 0s 115us/step
untrained model, accuracy : 1.4e+01%


이제 앞서 학습한 checkpoint로 부터 weight를 load해서 다시 평가해보면,

In [8]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 42us/step
Restored model, accuracy: 86.70%


### Checkpoint callback options

callback은 checkpoints 에 unique name을 지정하고, 체크포인트를 저장하는 빈도를 조절하는 options을 제공한다.

새로운 모델을 학습하고, 매 5 epoch 마다 다른 이름을 지정하여 checkpoint를 저장해보자.

In [9]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only=True,
                                                period=5) # Save weights every 5 epoch

model = create_model()
model.fit (train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


 이제 저장된 checkpoints를 살펴보자.

In [10]:
import pathlib

checkpoints = pathlib.Path(checkpoint_dir).glob("*.index")
checkpoints = sorted(checkpoints, key=lambda cp: cp.stat().st_mtime)
checkpoints = [cp.with_suffix('') for cp in checkpoints]
latest = str(checkpoints[-1])
checkpoints

[PosixPath('training_2/cp-0030.ckpt'),
 PosixPath('training_2/cp-0035.ckpt'),
 PosixPath('training_2/cp-0040.ckpt'),
 PosixPath('training_2/cp-0045.ckpt'),
 PosixPath('training_2/cp-0050.ckpt')]

In [11]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 87us/step
Restored model, accuracy: 87.80%


## <U>What are these files?</U>

위 code는 오직 학습된 weights 를 binary 포멧으로 저장하는 checkpoint-formatted file 로 weight를 저장한다. 
index 파일은 weights 가 어느 shard에 저장된 것인지 나타내 준다. 

## <U>Manually save weights</U>

수동으로 weight를 저장하는 방법은 `Model.save_weights` 를 호출하는 것이다.

In [12]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 91us/step
Restored model, accuracy: 87.80%


## <U>Save the entired model</U>

전체 모델은 weight values, model configuration, optimizer configuration 을 포함하는 하나의 파일로 저장할 수 있다. 이 방법을 사용하면 checkpoint 파일로 저장하고, 나중에 학습을 다시 이어서 진행할 수 있다. 

Keras에서 fully-functional model을 저장하는 것은 매우 유용하다. (이 파일을 Tensorflow.js 에서 가져와서 browser에서 학습하고 실행시킬 수 있다.)

Keras는 HDF5 표준을 사용한 기본 저장 포멧을 사용한다. 

In [13]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 458us/step - loss: 1.1971 - acc: 0.6600
Epoch 2/5
1000/1000 [==============================] - 0s 169us/step - loss: 0.4202 - acc: 0.8810
Epoch 3/5
1000/1000 [==============================] - 0s 167us/step - loss: 0.2771 - acc: 0.9290
Epoch 4/5
1000/1000 [==============================] - 0s 157us/step - loss: 0.2044 - acc: 0.9560
Epoch 5/5
1000/1000 [==============================] - 0s 180us/step - loss: 0.1520 - acc: 0.9680


저장한 H5 파일에서 모델을 recreate 해보자

In [14]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [15]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

1000/1000 [==============================] - 0s 113us/step
Restored model, accuracy: 85.40%


이 방법을 통해서 모델의 모든것을 저장할 수 있다. 
- weight values
- Model's configuration
- Optimizer configuration

Keras는 architecture inspecting을 통해서 모델을 저장할 수 있다. 그리고 현재 TensorFlow optimizer(tf.train) 은 저장할 수는 없다. 이를 사용할 때는 loading한 이후에 re-compile해야 한다. 